In [1]:
import warnings
warnings.filterwarnings('ignore')
import torch
import os
from pprint import pprint
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

import tool_pool as tp

dotenv_path = '/Users/minkyuramen/Desktop/project/env'
load_dotenv(dotenv_path)

api_key = os.getenv("SEMANTIC_SCHOLAR_API_KEY")
openai_key = os.getenv("OPENAI_API_KEY")

ModuleNotFoundError: No module named 'torch'

In [2]:
# # load model
# model = ChatOpenAI(model="gpt-3.5-turbo")
# load tool
tools = [tp.loadpaper, tp.recommendpaper, tp.loadfigure]
# load Agent prompt
prompt = hub.pull("hwchase17/openai-tools-agent")

In [3]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# loadpaper

In [ ]:
output = agent_executor.invoke({"input": "explain about the math expression about the Filtering api calls in the 3 page of the paper 'Language Models Can Teach Themselves to Use Tools' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

In [ ]:
output = agent_executor.invoke({"input": "summary the method of the paper 'Large Language Model Connected with Massive APIs' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

# recompendpaper

In [ ]:
output = agent_executor.invoke({"input": "recommend the future works of the paper 'Facilitating Large Language Models to Master 16000+ Real-world APIs' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

In [ ]:
output = agent_executor.invoke({"input": "Could you recommend previous works to read before this papaer : 'ToolLLM: Facilitating Large Language Models to Master 16000+ Real-world APIs' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

# loadfigure

In [ ]:
output = agent_executor.invoke({"input": "explain about the 'Figure 2: Pass Rate evaluation in ToolBench paper.' in the 3 page of the paper 'StableToolBench' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

## Memory in Agent

In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory

# prompt = hub.pull("hwchase17/react")
prompt = hub.pull("hwchase17/openai-tools-agent")
memory = ChatMessageHistory(session_id="test-session")
# message_history = RedisChatMessageHistory(
#     url="redis://127.0.0.1:6379/0", ttl=600, session_id="my-session"
# )

In [6]:
memory

InMemoryChatMessageHistory(messages=[])

In [7]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAI

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
output = agent_with_chat_history.invoke(
    {"input": "Summary the abstract of the paper 'EFFICIENT STREAMING LANGUAGE MODELS WITH ATTENTION SINKS'"},
    config={"configurable": {"session_id": "<test_session_id>"}},
)

In [ ]:
output = agent_with_chat_history.invoke(
    {"input": "저 논문에서 \n 토큰을 4번 넣으면서 확인하고 싶었던 게 뭐야?"},
    config={"configurable": {"session_id": "<test_session_id>"}},
)

In [ ]:
output = agent_with_chat_history.invoke(
    {"input": "이 논문의 후속 연구를 알려줘 최신순으로"},
    config={"configurable": {"session_id": "<test_session_id>"}},
)

### Real Use

In [8]:
from IPython.display import Markdown, display

In [9]:
def Paper_assistant(message, session_id = 'temp'):
    output = agent_with_chat_history.invoke(
        {"input": message},
        config={"configurable": {"session_id": session_id}},
    )
    display(Markdown(output['output']))
    return output

In [ ]:
Paper_assistant('PROMPTAGENT: STRATEGIC PLANNING WITH LANGUAGE MODELS ENABLES EXPERT-LEVEL PROMPT OPTIMIZATION 후속 논문 추천해줘')

In [10]:
Paper_assistant('"Learning to Evolve via Policy-Level Reflection and Optimization" 논문 초록 요약해')



> Entering new AgentExecutor chain...

Invoking: `loadpaper` with `{'title': 'Learning to Evolve via Policy-Level Reflection and Optimization'}`


Here is the title and section of the paper
title
Agent-Pro: Learning to Evolve via Policy-Level Reflection and Optimization
sections
          Abstract
  1 Introduction
  2 Problem Definition
  3 Methods
    3.1 Belief-aware Decision-Making Process
    3.2 Policy-Level Reflection
    3.3 DFS-based Policy Evolution
  4 Game: Blackjack
    4.1 Results
    4.2 Analyisis
  5 Game: Limit Texas Hold’em
    5.1 Results
    5.2 Analysis on Learning Process
    5.3 Analysis on Policy Evolution
  6 Conclusion
  Limitations
  References
  Appendix A Experiment Details
    A.1 LLMs
    A.2 Baselines
    A.3 Setups For Blackjack
    A.4 Detailed Setup For Texas Hold’em
  Appendix B Introduction of Two Games
    B.1 Blackjack
    B.2 Limit Texas Hold’em
    B.3 The Challenging of two Games
  Appendix C Complementary Experiments
    C.1 Whether Few-shot 

**Abstract Summary:**

Large Language Models (LLMs) are powerful problem solvers but are often designed for specific tasks using complex prompt engineering. These task-specific designs limit their ability to handle dynamic scenarios like large interactive games. To address this, the paper introduces Agent-Pro, an LLM-based agent that evolves its policy through interactions. Agent-Pro uses a dynamic belief generation and reflection process for policy evolution, focusing on policy-level rather than action-level reflection. It employs depth-first search for policy optimization, leading to continuous improvement. Evaluated in Blackjack and Texas Hold’em, Agent-Pro outperforms standard LLMs and specialized models, demonstrating its ability to learn and adapt in complex environments.

{'input': '"Learning to Evolve via Policy-Level Reflection and Optimization" 논문 초록 요약해',
 'chat_history': [],
 'output': '**Abstract Summary:**\n\nLarge Language Models (LLMs) are powerful problem solvers but are often designed for specific tasks using complex prompt engineering. These task-specific designs limit their ability to handle dynamic scenarios like large interactive games. To address this, the paper introduces Agent-Pro, an LLM-based agent that evolves its policy through interactions. Agent-Pro uses a dynamic belief generation and reflection process for policy evolution, focusing on policy-level rather than action-level reflection. It employs depth-first search for policy optimization, leading to continuous improvement. Evaluated in Blackjack and Texas Hold’em, Agent-Pro outperforms standard LLMs and specialized models, demonstrating its ability to learn and adapt in complex environments.'}

In [11]:
Paper_assistant('논문 Method의 각 요소를 요약해줘')



> Entering new AgentExecutor chain...

Invoking: `loadpaper` with `{'title': 'Learning to Evolve via Policy-Level Reflection and Optimization', 'sections': []}`


Section list was incorrect.
Here is the title and section of the paper
title
Agent-Pro: Learning to Evolve via Policy-Level Reflection and Optimization
sections
          Abstract
  1 Introduction
  2 Problem Definition
  3 Methods
    3.1 Belief-aware Decision-Making Process
    3.2 Policy-Level Reflection
    3.3 DFS-based Policy Evolution
  4 Game: Blackjack
    4.1 Results
    4.2 Analyisis
  5 Game: Limit Texas Hold’em
    5.1 Results
    5.2 Analysis on Learning Process
    5.3 Analysis on Policy Evolution
  6 Conclusion
  Limitations
  References
  Appendix A Experiment Details
    A.1 LLMs
    A.2 Baselines
    A.3 Setups For Blackjack
    A.4 Detailed Setup For Texas Hold’em
  Appendix B Introduction of Two Games
    B.1 Blackjack
    B.2 Limit Texas Hold’em
    B.3 The Challenging of two Games
  Appendix C Complem

### Methods Summary

#### 3.1 Belief-aware Decision-Making Process
- **Inspiration**: Draws from the Theory of Mind (ToM), where humans form beliefs about themselves (self-belief) and the environment (social-belief) to guide decisions.
- **Policy Definition**: The policy of an LLM-based agent involves complex prompts covering task rules, strategies, and output formats.
- **Dynamic Belief Generation**: Agent-Pro generates dynamic beliefs about itself and opponents in natural language, updating these beliefs in each decision-making cycle.
- **Decision Making**: Actions are predicted based on the latest beliefs, aiming for coherent and consistent actions.
- **Objective**: To find an optimal policy that maximizes the agent's score.

#### 3.2 Policy-Level Reflection
- **Initial Capabilities**: Agent-Pro starts with a basic policy and dynamic belief for game exploration.
- **Reflection Mechanism**: Focuses on the rationality of beliefs and underlying behavioral policies rather than individual actions.
- **Belief Calibration**: Agent-Pro examines the correctness, consistency, and rationality of its beliefs based on final results and reflects on the reasons for failure.
- **Instruction Generation**: Summarizes reflections into Behavioral Guidelines and World Modeling to calibrate incorrect beliefs and improve policy performance.
- **Verification**: Incorporates new instructions into the prompt and replays the same game to verify improvements. If the score improves, the instructions are retained; otherwise, they are regenerated or discarded after three retries.

#### 3.3 DFS-based Policy Evolution
- **Policy Evaluation**: Evaluates new policies in novel game scenarios to ensure generalizability, eliminating the influence of random factors by generating multiple game combinations.
- **Evaluation Metrics**: Calculates the average score over all combinations to represent true capabilities and assesses both absolute gains and relative gains to the strongest opponent.
- **Policy Search**: Uses depth-first search (DFS) to explore and update policies. Generates multiple candidate policies and selects the one with the highest performance improvement. If no improvement is found, backtracks to consider sibling nodes and continues the search.

These methods collectively enable Agent-Pro to evolve its policy through dynamic belief generation, reflection, and optimization, allowing it to adapt and improve in complex interactive environments.

{'input': '논문 Method의 각 요소를 요약해줘',
 'chat_history': [HumanMessage(content='"Learning to Evolve via Policy-Level Reflection and Optimization" 논문 초록 요약해'),
  AIMessage(content='**Abstract Summary:**\n\nLarge Language Models (LLMs) are powerful problem solvers but are often designed for specific tasks using complex prompt engineering. These task-specific designs limit their ability to handle dynamic scenarios like large interactive games. To address this, the paper introduces Agent-Pro, an LLM-based agent that evolves its policy through interactions. Agent-Pro uses a dynamic belief generation and reflection process for policy evolution, focusing on policy-level rather than action-level reflection. It employs depth-first search for policy optimization, leading to continuous improvement. Evaluated in Blackjack and Texas Hold’em, Agent-Pro outperforms standard LLMs and specialized models, demonstrating its ability to learn and adapt in complex environments.')],
 'output': "### Methods Summar